In [0]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu
from elopy.elo import Elo
import datetime

# Read recipe inputs
spreadspoke_scores = dataiku.Dataset("nfl_scores")
games_df = spreadspoke_scores.get_dataframe()

In [0]:
games_df.head()

## Defining start date for when the Elo ranking should begin

In [0]:
#If setting 8/1/2022 (2022,8,1) in this syntax, then each team starts with an Elo rating of 1600
#at the start of the 2022-2023 regular season
start = datetime.datetime(2022,8,1)
qry = f'schedule_date>\'{start}+00:00\' & score_home.notna()'
hist_games_df = games_df.query(qry, engine='python')

In [0]:
len(hist_games_df)

## Establishes ELO Rating for teams at the start date and updates their rating through all games that have been played - games that have a value for score_home

In [0]:
teams = {}
stadium_location = {}

hca_value = 100
for team in hist_games_df['team_home'].unique():
    teams[team] = Elo(start_elo=1600, k=20, hca = hca_value)

for idx, row in hist_games_df.sort_values(['schedule_date']).iterrows():
#The if loop is evaluating who won historical games -home team or away team.
#It also is broken down to consider if the game was played at a neutral site
#If at a neutral site, the "home team" doesn't get the Elo rating equal to hca_value
    if row['score_home'] > row['score_away'] and row['stadium_neutral'] == True:
        winby = row['score_home'] - row['score_away']
        teams[row['team_home']].play_game(teams[row['team_away']], winby, is_home=False)
    elif row['score_home'] > row['score_away'] and row['stadium_neutral'] == False:
        winby = row['score_home'] - row['score_away']
        teams[row['team_home']].play_game(teams[row['team_away']], winby, is_home=True)
    else:
        winby = row['score_away'] - row['score_home']
        teams[row['team_away']].play_game(teams[row['team_home']], winby, is_home=False)

    hist_games_df.at[idx, 'home_elo'] = teams[row['team_home']].elo
    hist_games_df.at[idx, 'away_elo'] = teams[row['team_away']].elo

## Record the team ELO Ratings

In [0]:
results = []
for team in hist_games_df['team_home'].unique():
    print(team, teams[team].elo)
    results.append({'team': team, 'rating': teams[team].elo})

## Grab the upcoming games

In [0]:
#days_ahead lets you pick how many days ahead to go from the defined start date - normally would look ahead 1 week
#but with a 2 week gap for Super Bowl, you need to go ahead 2 weeks from 1/31/2023 (defined in start below)
days_ahead = 14

start = datetime.datetime(2023,1,31) + datetime.timedelta(days=days_ahead)
qry = f'schedule_date<\'{start}+00:00\' & not score_home.notna()'
upcoming_games_df = games_df.query(qry, engine='python')

## Computing the odds of a home team win

In [0]:
tw_pred = []
#This section has an if statement again to make sure that the "home team" is actually playing on their home field
#and should get the Home Field Advantage boost or not.

for idx, row in upcoming_games_df.iterrows():
    if row['stadium_neutral'] == True:
        tw_pred.append({'home_team':row['team_home'],
                        'away_team':row['team_away'],
                        'date': row['schedule_date'].strftime("%Y-%m-%d"),
                        'spread': 0-(teams[row['team_home']].point_spread(teams[row['team_away']], is_home=False)),
                        'home_odds':teams[row['team_home']].win_probs(teams[row['team_away']], is_home=False)})
    elif row['stadium_neutral'] == False:
        tw_pred.append({'home_team':row['team_home'],
                        'away_team':row['team_away'],
                        'date': row['schedule_date'].strftime("%Y-%m-%d"),
                        'spread': 0-(teams[row['team_home']].point_spread(teams[row['team_away']], is_home=True)),
                        'home_odds':teams[row['team_home']].win_probs(teams[row['team_away']], is_home=True)})

In [0]:
tw_pred

In [0]:
# Write ELO outputs
elo_results = dataiku.Dataset("elo_results")
elo_results.write_with_schema(pd.DataFrame.from_dict(results))

In [0]:
upcoming_results = dataiku.Dataset("upcoming_game_probabilities")
upcoming_results.write_with_schema(pd.DataFrame.from_dict(tw_pred))